In [71]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

# I. Classification

Dans une première partie, nous allons nous intéresser à la classification. On va importer des datasets de sklearn et entraîner notre modèle de XGBOOST sur de la classification binaire et multi-classes et le comparer avec deux modèles d'arbres de décision.

### I.1 Importer les datasets

In [79]:
from sklearn.datasets import load_iris, fetch_covtype, load_breast_cancer  
from sklearn.model_selection import train_test_split

# Charger un jeu de données Iris pour classification
iris = load_iris()
X, Y = iris.data, iris.target

covertype = fetch_covtype()
X, Y = covertype.data, covertype.target
Y = Y - 1   # car les classes dans cette dataset commencent de 1 et pas de 0 

cancer = load_breast_cancer()   # Pour la classification binaire
X, Y = cancer.data, cancer.target

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((455, 30), (114, 30))

### I.2 Entraîner deux modèles d'arbres de décision

In [73]:
from   sklearn.tree    import DecisionTreeClassifier
from   sklearn.metrics import accuracy_score, f1_score
from   sklearn         import tree
import timeit

Xchoix_train = X_train
Ychoix_train = Y_train
Xchoix_test  = X_test
Ychoix_test  = Y_test
# fnames       = iris.columns


gini_stats          = []
entropy_stats       = []
gini_classifieur    = DecisionTreeClassifier(criterion='gini'   , random_state=42)
entropy_classifieur = DecisionTreeClassifier(criterion='entropy', random_state=42)

# ============ GINI ====================
# ............ Entraînement ............
temps_debut = timeit.default_timer()
gini_classifieur.fit(Xchoix_train, Ychoix_train)
gini_stats.append(timeit.default_timer() - temps_debut)
# ..... Evaluation entrainement ........
Ychoix_pred = gini_classifieur.predict(Xchoix_train)
gini_stats.append(accuracy_score(Ychoix_train, Ychoix_pred))
gini_stats.append(f1_score(Ychoix_train, Ychoix_pred, average='micro'))
# ................ Test ................
temps_debut = timeit.default_timer()
Ychoix_pred = gini_classifieur.predict(Xchoix_test)
gini_stats.append(timeit.default_timer() - temps_debut)
# ........... Evaluation test ...........
gini_stats.append(accuracy_score(Ychoix_test, Ychoix_pred))
gini_stats.append(f1_score(Ychoix_test, Ychoix_pred, average='micro'))

# =========== Entropy ==================
# ............ Entraînement ............
temps_debut = timeit.default_timer()
entropy_classifieur.fit(Xchoix_train, Ychoix_train)
entropy_stats.append(timeit.default_timer() - temps_debut)
# ..... Evaluation entrainement ........
Ychoix_pred = entropy_classifieur.predict(Xchoix_train)
entropy_stats.append(accuracy_score(Ychoix_train, Ychoix_pred))
entropy_stats.append(f1_score(Ychoix_train, Ychoix_pred, average='micro'))
# ................ Test ................
temps_debut = timeit.default_timer()
Ychoix_pred = entropy_classifieur.predict(Xchoix_test)
entropy_stats.append(timeit.default_timer() - temps_debut)
# ........... Evaluation test ...........
entropy_stats.append(accuracy_score(Ychoix_test, Ychoix_pred))
entropy_stats.append(f1_score(Ychoix_test, Ychoix_pred, average='micro'))

print('Fin')

Fin


### I.3 Entraîner un modèle XGBOOST

In [74]:
import xgboost as xgb

# Définir les paramètres du modèle
# params = {
#     'objective': 'multi:softmax',  # Fonction objective pour la classification multi-classe
#     'num_class': 7,  # Nombre de classes dans le jeu de données 
#     'max_depth': 4  # Profondeur maximale de l'arbre
# }
# params = {
#     'objective': 'binary:logistic',  # Fonction objective pour la classification binaire
#     'num_class': 2, 
#     'max_depth': 3
# }
XGBOOST_stats = []

# Créer le modèle de classification (avec les paramètres par défaut)
clf = xgb.XGBClassifier()


# ================= Entraînement =====================
temps_debut = timeit.default_timer()
# Entraîner le modèle
clf.fit(X_train, Y_train)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)
Y_pred = clf.predict(X_train)
XGBOOST_stats.append(accuracy_score(Y_train, Y_pred))
XGBOOST_stats.append(f1_score(Y_train, Y_pred, average='micro'))

# ===================== Test ========================
temps_debut = timeit.default_timer()
# Faire des prédictions sur l'ensemble de test
Y_pred = clf.predict(X_test)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)
XGBOOST_stats.append(accuracy_score(Y_test, Y_pred))
XGBOOST_stats.append(f1_score(Y_test, Y_pred, average='micro'))

In [75]:
pd.DataFrame({
    'Criteres' : ['Temps Entrainement', 'Accuracy Entrainement', 'F1 Entrainement', 'Temps Test', 'Accuracy Test', 'F1 Test'],
    'Entropie' : entropy_stats,
    'Gini'     : gini_stats,
    'XGBOOST'  : XGBOOST_stats
})

,Criteres,Entropie,Gini,XGBOOST
0,Temps Entrainement,0.021726,0.026059,0.108158
1,Accuracy Entrainement,1.000000,1.000000,1.000000
2,F1 Entrainement,1.000000,1.000000,1.000000
3,Temps Test,0.000228,0.000451,0.000718
4,Accuracy Test,0.947368,0.947368,0.956140
5,F1 Test,0.947368,0.947368,0.956140


# II. Régression

Dans cette seconde partie, nous allons entraîner un modèle de XGBOOST pour la régression et le comparer avec le modèle de régression linéaire en utilisant une dataset de Sklearn comportant les prix des maisons de California.

### II.1 Importer la dataset

In [77]:
from sklearn.datasets import fetch_california_housing

houses = fetch_california_housing()
X, Y = houses.data, houses.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((16512, 8), (4128, 8))

### II.2 Entraîner un modèle de régression linéaire

In [84]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline      import make_pipeline
from sklearn.metrics import mean_squared_error

lineaire_stats = []

reg_lineaire = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

temps_debut = timeit.default_timer()
# Entraîner le modèle de régression linéaire
reg_lineaire.fit(X_train, Y_train)
lineaire_stats.append(timeit.default_timer() - temps_debut)

temps_debut = timeit.default_timer()
# Prédire
Y_pred_lineaire = reg_lineaire.predict(X_test)
lineaire_stats.append(timeit.default_timer() - temps_debut)

# Calculer l'erreur quadratique moyenne (MSE)
mse_lineaire = mean_squared_error(Y_test, Y_pred_lineaire)
lineaire_stats.append(mse_lineaire)
print(f"MSE: {mse_lineaire}")

MSE: 0.06410886247029467


### II.3 Entraîner un modèle XGBOOST

In [87]:
# Définir les paramètres du modèle
params = {
    'objective': 'reg:squarederror',  # Fonction objective pour la régression
    'max_depth': 3  # Profondeur maximale de l'arbre
}

XGBOOST_stats = []

# Créer le modèle de régression XGBOOST avec les paramètres, changer la profondeur change le MSE, plus profond mieux c'est mais prend plus de temps
reg_XGBOOST = xgb.XGBRegressor(**params)

temps_debut = timeit.default_timer()
# Entraîner le modèle
reg_XGBOOST.fit(X_train, Y_train)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)

temps_debut = timeit.default_timer()
# Faire des prédictions sur l'ensemble de test
Y_pred_XGBOOST = reg_XGBOOST.predict(X_test)
XGBOOST_stats.append(timeit.default_timer() - temps_debut)

# Calculer l'erreur quadratique moyenne (MSE)
mse_XGBOOST = mean_squared_error(Y_test, Y_pred_XGBOOST)
XGBOOST_stats.append(mse_XGBOOST)
print(f"MSE: {mse_XGBOOST}")


MSE: 0.03559795217495412


In [88]:
pd.DataFrame({
    'Criteres' : ['Temps Entrainement','Temps Test', 'MSE'],
    'Linéaire' : lineaire_stats,
    'XGBOOST'  : XGBOOST_stats
})

,Criteres,Linéaire,XGBOOST
0,Temps Entrainement,0.010386,0.213670
1,Temps Test,0.001254,0.001098
2,MSE,0.064109,0.035598
